In [1]:
with open("hf-token.txt", "r") as f:
    token = f.read()

LF = "\n"

In [2]:
%%capture
!pip install flash-attn --no-build-isolation

import torch
from transformers import pipeline
import pandas as pd

print(torch.mps.is_available())

In [3]:
from transformers import pipeline

# deepseek-ai/deepseek-llm-7b-base: 7m 23s for loading (no feedback in the last minute)
# deepseek-ai/DeepSeek-R1-Distill-Qwen-14B: 13m 23s
pipeline = pipeline(
    task="text-generation", 
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", 
    device = 0, # 0 for gpu, -1 for cpu
    torch_dtype = "auto",
    trust_remote_code = True,
    token = token
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
cls_ref = {
    0 : "Algebra",
    1 : "Geometry & Trigonometry",
    2 : "Calculus & Analysis",
    3 : "Probability & Statistics",
    4 : "Number Theory",
    5 : "Combinatorics & Discrete Math",
    6 : "Linear Algebra",
    7 : "Abstract Algebra & Topology"
}

def gen_prompt(problem, cls_ref):

    txt = """You are a math expert. Your job is to read math problems and use your expertise to classify them 
into the appropriate subcategories. Please select the subcategory below that best describes 
the following problem:"""

    txt += LF * 2 + '"' + problem + '"' + LF * 2

    for k,v in cls_ref.items():
        txt += f"({k}) {v} {LF}"

    # txt += f"Please provide an answer in the form: (<n>) <subject>"

    return txt

In [5]:
problem = "A solitaire game is played as follows.  Six distinct pairs of matched tiles are placed in a bag.  The player randomly draws tiles one at a time from the bag and retains them, except that matching tiles are put aside as soon as they appear in the player's hand.  The game ends if the player ever holds three tiles, no two of which match; otherwise the drawing continues until the bag is empty.  The probability that the bag will be emptied is $p/q,\,$ where $p\,$ and $q\,$ are relatively prime positive integers.  Find $p+q.\,$ "
problem = "2. The school table tennis championship was held according to the Olympic system. The winner won 6 matches. How many participants in the championship won more matches than they lost? (In the first round of the championship held according to the Olympic system, participants are paired. Those who lost the first game are eliminated from the championship, while those who won in the first round are paired again to play the second round. The losers are eliminated again, the winners are paired for the third round, and so on, until only one champion remains. It is known that in each round of our championship, a pair was found for each participant.)"
prompt = gen_prompt(problem, cls_ref)

In [6]:
df_tr = pd.read_csv("train-data/train.csv")

In [7]:
llm_ref = {
    "0" : 0,
    "geo" : 1,
    "trig" : 1,
    "1" : 1,
    "calc" : 2,
    "analy" : 2,
    "2" : 2,
    "probab" : 3,
    "stat" : 3,
    "3" : 3,
    "num" : 4,
    "theory" : 4,
    "4" : 4,
    "combin" : 5,
    "disc" : 5,
    "5" : 5,
    "line" : 6,
    "6" : 6,
    "abstr" : 7,
    "topol" : 7,
    "7" : 7,
    "algebra" : 0
}

def interpret_llm(output):
    ''' attempts to convert the llm's answer to the correct format '''

    ans = 8

    for k,v in llm_ref.items():
        if k in output: 
            ans = v
            break

    if ans == 8:
        return "(8) Err"
    
    else:
        return f"({ans}) {cls_ref[ans]}"

In [8]:
# deepseek 7b: 5s per query, correct answer around 40% of the time
# deepseek distill qwen 14b: 

for i,(q,a) in enumerate(zip(df_tr.Question, df_tr.label)):

    prompt = gen_prompt(q, cls_ref)

    print(f"\n== question {i}, label={a} ==\n")
    print(q)

    output = pipeline(prompt, max_new_tokens = 150)[0]["generated_text"][-100:]
    print(output)
    ans = interpret_llm(output)
    print(f"LLM response: {ans}")



== question 0, label=3 ==

A solitaire game is played as follows.  Six distinct pairs of matched tiles are placed in a bag.  The player randomly draws tiles one at a time from the bag and retains them, except that matching tiles are put aside as soon as they appear in the player's hand.  The game ends if the player ever holds three tiles, no two of which match; otherwise the drawing continues until the bag is empty.  The probability that the bag will be emptied is $p/q,\,$ where $p\,$ and $q\,$ are relatively prime positive integers.  Find $p+q.\,$ 


KeyboardInterrupt: 